In [2]:
import pandas as pd
import numpy as np

# Dataset

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [4]:
display(train.head())
display(train.describe())
display(train.Survived.value_counts())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Survived
0    549
1    342
Name: count, dtype: int64

In [5]:
display(train.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# Extraer género del nombre

In [6]:
train.Name[-train.Name.str.contains('Mr|Mrs|Miss|Master|Don.|Dr.|Ms.|Mme.|Rev.|Col.|Major.|Countess.|Capt.|Jonkheer|Mlle.', case=False)]

Series([], Name: Name, dtype: object)

In [7]:
# Extraer el título con una expresión regular
train['Title'] = train['Name'].str.extract(r',\s*([^\.]+)\.', expand=False)
# contar titles
display(train.Title.value_counts())
#TODO: Agrupar títulos poco frecuentes

Title
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: count, dtype: int64

In [8]:
train['FamilySize'] = train['SibSp'] + train['Parch']
#isalone si esta sin familia
train['IsAlone'] = train.FamilySize == 0
#pasar Sex a numero, 1 si es hombre 0 si no
train['sex_binarized'] = np.where(train.Sex == 'male', 1, 0)
#binarizar Embarked


In [9]:
train.Embarked.value_counts(dropna = False)

Embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64

In [10]:
train[train.Embarked.isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone,sex_binarized
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,Miss,0,True,0
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,Mrs,0,True,0


In [11]:
train['Fare'].std()

np.float64(49.6934285971809)

In [12]:
train.loc[(train.Pclass == 1) & (train.Fare.between(80-5, 80+5)),'Embarked'].value_counts()

Embarked
C    16
S    13
Name: count, dtype: int64